##**Mendeteksi Outlier pada Iris Dataset dengan Metode LOF (Local Outlier Factor)**

# Local Outlier Factor (LOF)

Local Outlier Factor (**LOF**) adalah algoritma **deteksi outlier berbasis kepadatan (density-based)** yang membandingkan **kepadatan lokal** suatu titik dengan tetangganya. Jika suatu titik memiliki **kepadatan yang jauh lebih rendah dibandingkan tetangganya**, maka titik tersebut dianggap sebagai **outlier**.

---

## **Tahapan LOF**

### **1. Pilih $k$ (Jumlah Tetangga Terdekat)**
LOF bergantung pada **$k$-nearest neighbors (KNN)**.  
- **$k$ terlalu kecil** → Bisa salah mendeteksi outlier karena terlalu sensitif.  
- **$k$ terlalu besar** → Bisa kehilangan outlier karena efek perataan.  
- Biasanya $k$ dipilih antara **10 hingga 20** tergantung pada dataset.  

---

### **2. Hitung Jarak ke Tetangga Terdekat ($k$-Distance)**  
Untuk setiap titik $A$, cari **$k$ tetangga terdekatnya**.  
Hitung jarak titik $A$ ke masing-masing tetangganya.  
Jarak ke tetangga ke-$k$ disebut **$k$-distance**.

---

### **3. Hitung Reachability Distance**  
Reachability distance mengukur **seberapa jauh titik $A$ dari tetangganya**, tetapi dibatasi oleh jarak minimum yang dimiliki tetangganya.  

\[$
\text{reachability-distance}(A, B) = \max(k\text{-distance}(B), d(A, B))
$\]

di mana:  
- $d(A, B)$ adalah jarak antara titik $A$ dan $B$.  
- $k\text{-distance}(B)$ adalah jarak ke-$k$ dari tetangga $B$.  

---

### **4. Hitung Kepadatan Lokal (Local Reachability Density - LRD)**  
Kepadatan lokal suatu titik dihitung sebagai kebalikan dari rata-rata **reachability distance** dengan $k$ tetangganya.  

\[$
\text{LRD}(A) = \frac{k}{\sum\limits_{B \in k\text{-neighbors}(A)} \frac{\text{reachability-distance}(A, B)}{k}}
$\]

 **Intinya:**  
- Jika nilai **LRD tinggi** → Titik ini berada di daerah dengan **kepadatan tinggi**.  
- Jika nilai **LRD rendah** → Titik ini berada di daerah dengan **kepadatan rendah (kemungkinan outlier)**.

---

### **5. Hitung Local Outlier Factor (LOF)**
LOF membandingkan kepadatan titik dengan kepadatan tetangganya.  

\[$
\text{LOF}(A) = \frac{\sum\limits_{B \in k\text{-neighbors}(A)} \frac{\text{LRD}(B)}{\text{LRD}(A)}}{k}
$\]

**Interpretasi LOF Score:**  
- **LOF $\approx$ 1** → Titik ini mirip dengan tetangganya (**bukan outlier**).  
- **LOF $> 1$** → Titik ini lebih jarang dibandingkan tetangganya (**mungkin outlier**).  
- **LOF $\gg 1$** → Titik ini **sangat jarang dibandingkan tetangganya** (**kemungkinan besar outlier**).  

---



## **Perhitungan LOF secara Manual**
1. Pilih nilai k
Kita pilih k = 2 (jumlah tetangga terdekat).

2. Hitung k-distance
Untuk setiap titik, hitung jarak Euclidean dengan titik lainnya (contohnya hanya untuk kolom numerik: sepal_length, sepal_width, petal_length, petal_width).

Contoh untuk titik 0:

Jarak ke titik 1: $$\sqrt{(1.4 - 14.0)^2 + (0.2 - 2.0)^2 + (5.1 - 40.9)^2 + (3.5 - 30.0)^2} \approx 47.8$$

Jarak ke titik 2: $$\sqrt{(1.4 - 1.3)^2 + (0.2 - 0.2)^2 + (5.1 - 4.7)^2 + (3.5 - 3.2)^2} \approx 0.54$$

Lakukan perhitungan serupa untuk semua titik, dan catat dua jarak terkecil setiap titik (karena k = 2).

3. Hitung reachability-distance
Untuk setiap titik A, dengan tetangganya B: $$\text{reachability-distance}(A, B) = \max(\text{k-distance}(B), d(A, B))$$

Contoh untuk titik 0:

Misalkan tetangga terdekat adalah titik 2 dan 3, hitung nilai seperti di atas.

4. Hitung Local Reachability Density (LRD)
LRD dihitung sebagai: $$\text{LRD}(A) = \frac{k}{\sum_{B \in \text{k-neighbors}(A)} \text{reachability-distance}(A, B)}$$

Contoh untuk titik 0: LRD dihitung dengan membagi 2 dengan rata-rata dari reachability-distance ke tetangga.

5. Hitung Local Outlier Factor (LOF)
LOF dibandingkan dengan tetangga: $$\text{LOF}(A) = \frac{\sum_{B \in \text{k-neighbors}(A)} \frac{\text{LRD}(B)}{\text{LRD}(A)}}{k}$$

Interpretasi:

LOF ≈ 1: Titik normal.

LOF > 1: Titik mungkin outlier.

LOF ≫ 1: Titik adalah outlier.

Hasil Manual (Estimasi):

Titik dengan ID 2 memiliki nilai LOF jauh lebih tinggi dibandingkan lainnya (karena nilai-nilai seperti 14.0 atau 40.9 sangat ekstrem), sehingga ID 2 adalah outlier.

## **Implementasi kode Deteksi Outlier dengan metode LOF**

In [ ]:
%pip install pymysql
%pip install psycopg2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.8 MB/s eta 0:00:00


setelah melakukan instalasi kedua library connector database tersebut, kita akan melakukan penggabungan data dari kedua database, jika anda ingin penjelasan detail silahkan kembali ke halaman sebelumnya

In [ ]:

import pymysql
import psycopg2
import numpy as np
import pandas as pd

def get_mysql_data():
    conn = pymysql.connect(
        host="pendatviomysql-39-projectvioo.h.aivencloud.com",
        user="avnadmin",
        password="AVNS_nnGCVuLriFaCit_hSPr",
        database="myiris",
        port=20305
    )
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM irismysql")  # Select all records from the table
    data = cursor.fetchall()  # Fetch all rows
    conn.close()
    # Convert to Python list
    data_list = [list(row) for row in data]
    # Convert to NumPy array
    data_numpy = np.array(data_list)
    return data_list



def get_pg_data():
    conn = psycopg2.connect(
        host="pg-30810f3a-projectvioo.h.aivencloud.com",
        user="avnadmin",
        password="AVNS_zzD9DhqapmhcWhqwe5C",
        database="defaultdb",
        port=20305
    )
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM iris_post")  # Select all records from the table
    data = cursor.fetchall()  # Fetch all rows
    cursor.close()
    conn.close()

    # Convert to Python list
    data_list = [list(row) for row in data]

    # Convert to NumPy array
    data_numpy = np.array(data_list)

    return data_list

columns = ['id', 'Class', 'sepal_length', 'sepal_width']
data_mysql = pd.DataFrame(get_mysql_data(), columns=columns)

columns = ['id', 'Class', 'petal_length', 'petal_width']
data_pg = pd.DataFrame(get_pg_data(), columns=columns)

df_merged = pd.merge(data_mysql, data_pg, on=["id", "Class"], how="inner")
print(df_merged)




      id           Class  sepal_length  sepal_width  petal_length  petal_width
0      1     Iris-setosa           1.4          0.2           5.1          3.5
1      2     Iris-setosa          14.0          2.0          40.9         30.0
2      3     Iris-setosa           1.3          0.2           4.7          3.2
3      4     Iris-setosa           1.5          0.2           4.6          3.1
4      5     Iris-setosa           1.4          0.2           5.0          3.6
..   ...             ...           ...          ...           ...          ...
146  147  Iris-virginica           5.0          1.9           6.3          2.5
147  148  Iris-virginica           5.2          2.0           6.5          3.0
148  149  Iris-virginica           5.4          2.3           6.2          3.4
149  150  Iris-virginica           5.1          1.8           5.9          3.0
150  151             ???           5.1          3.2           5.8          1.0

[151 rows x 6 columns]


In [ ]:
import pandas as pd
from sklearn.neighbors import LocalOutlierFactor

# Ambil data fitur numerik tanpa kolom 'class'
feature_columns = ["petal_length", "petal_width", "sepal_length", "sepal_width"]
data_values = df_merged[feature_columns].values

# Inisialisasi model LOF
clf = LocalOutlierFactor(n_neighbors=90)
label = clf.fit_predict(data_values)

# Tambahkan hasil label ke dataframe
df_merged["outlier_label"] = label

# Cetak hasil dengan ID dan class
print(df_merged.to_string(index=False))

num_outliers = (label == -1).sum()
print(f"\nJumlah outlier: {num_outliers}")

outliers = df_merged[df_merged["outlier_label"] == -1]
print("\nData Outlier:")
print(outliers.to_string(index=False))

 id           Class  sepal_length  sepal_width  petal_length  petal_width  outlier_label
  1     Iris-setosa           1.4          0.2           5.1          3.5              1
  2     Iris-setosa          14.0          2.0          40.9         30.0             -1
  3     Iris-setosa           1.3          0.2           4.7          3.2              1
  4     Iris-setosa           1.5          0.2           4.6          3.1              1
  5     Iris-setosa           1.4          0.2           5.0          3.6              1
  6     Iris-setosa           1.7          0.4           5.4          3.9              1
  7     Iris-setosa           1.4          0.3           4.6          3.4              1
  8     Iris-setosa           1.5          0.2           5.0          3.4              1
  9     Iris-setosa           1.4          0.2           4.4          2.9              1
 10     Iris-setosa           1.5          0.1           4.9          3.1              1
 11     Iris-setosa  